In [1]:
# TODOO - Real state trajectory (provided) NORMALEMENT OK
#       - Filtered states (result of the update step)
#       - 95% confidence interval (+- 1.96 sigma) around the filtered states
#       - RMS Deviation as a function of k

# We Have - true_state = real state trajectory
#         - input = u
#         - measured output = y = x2 measured

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

true_x1 = pd.read_csv('Q1/True_state_x1_linear_case.txt',sep=" ", header=0)['True']
true_x2 = pd.read_csv('Q1/True_state_x2_linear_case.txt',sep=" ", header=0)['True']
command = pd.read_csv('Q1/Input_linear_case.txt',sep=" ", header=0)['input']
mes = pd.read_csv('Q1/Measured_output_linear_case.txt',sep=" ", header=0)['output']

time = np.linspace(0,100,1000)
noise = np.random.normal(0,0.0125) # faut-il ajouter du noise à y ?

plt.figure(figsize=(25,8))
plt.plot(time,true_x1, label="True_x1")
plt.plot(time,true_x2, label="True_x2")
plt.plot(time,command, label="u")
plt.plot(time,mes, label="y")
plt.legend(loc="upper right")
plt.show()





FileNotFoundError: [Errno 2] No such file or directory: 'Q1/True_state_x1_linear_case.txt'

In [ ]:
dt=0.1 #seconds
Nsteps = 1000 


# Matrices

# Lien step en fonction du step d'avant
A = np.array([[0.9512, 0],   
              [0.0476, 0.9512]])  

# Lien step en fct de la commande
B = np.array([[0.0975],
              [0.0024]])

# Lien mesure en fct de l'état
C = np.array([0, 1])
CT = np.array([[0],
              [1]])

# Covariance de l'état initial
P0 = np.array([[1, 0],
               [0, 1]])

# Process noise (=W)
Q = np.array([[0.009506, 0.000234],   
              [0.000234, 0.009506]])      

# Measurement noise (1x1)
R = 0.0125


# Initial states
x1_0 = 5 + np.random.normal(0, 1)
x2_0 = 5 + np.random.normal(0, 1)

init = np.array([[x1_0],
                [x2_0]])

print(B*C)


# Questions en suspens

# W et V? quelle dimension ? taille v = 1 et Q = 2*2 pcq W = 2*1
# Ajouter du bruit au mesures y ? non, c'est V qui le fait
# initial state + bruit ou juste bruit ? oui, du bruit car P0 est 1 et 0
# Quelle taille pour K et P ?  K = 2*1 P = 2.2

# Précision muk = muk tilde (eq 3) -> à utiliser pour x
#xk = muk


In [2]:
def Kalman_predictor(A, B, Q, init, P0, C, CT, R, Measurements, command, Nsteps):
    
    # Output : 
    # - Predicted_states, a Nsteps x 2 array containing the states at all time indexes
    
    K = np.empty((Nsteps, 2, 1), dtype=float)
    P = np.empty((Nsteps, 2, 2), dtype=float)
    mu = np.empty((Nsteps, 2, 1), dtype=float)
    
    P[0, :, :] = P0
    mu[0, :] = init;
    
    for k in range(Nsteps - 1):
        bruit = np.random.normal(0,R)
        
        K[k, :, :] = (A @ P[k, :, :] @ A.T + Q) @ CT * 1/(C @ (A @ P[k, :, :] @ A.T + Q) @ CT + R)
        
        P[k+1, :, :] =  (np.eye(2) - K[k, :, :] * C) @ (A @ P[k, :, :] @ A.T + Q) 
        
        mu_tild = A @ mu[k, :, :] + B * command[k]  # (2*1)
        mu[k+1, :, :] = mu_tild + K[k, :, :] * (Measurements[k] - C @ mu_tild)
      
    return mu

In [3]:
pred_states = Kalman_predictor(A, B, Q, init, P0, C, CT, R, mes, command, Nsteps)


plt.figure(figsize=(25,8))
plt.plot(time,true_x1, label="True_x1")
plt.plot(time,true_x2, label="True_x2")
plt.plot(time, pred_states[:,0], ".",label="Predicted_x1")
plt.plot(time, pred_states[:,1], ".",label="Predicted_x2")
plt.legend(loc="upper right")
plt.show()


NameError: name 'A' is not defined

In [ ]:
# Ensemble Kalman filter 

# Reprendre le code du predictor et modifier l'update step avec les formules
# de l'énoncé pour K et x (mu = x)
# reprendre P dans le doc ue je t'ai envoyé 